# Test hypotézy

S 95% jistotou ověřte hypotézu:  
Pokud viník nehody byl pod silným vlivem alkoholu,  
došlo častěji k těžkým zdravotním následkům.

Hypotézy nelze obecně potvrdit, jen s určitou jistotou vyvrátit, proto  
se pokusíme vyvrátit hypotézu, že naopak silný vliv alkoholu závažnost  
zranění neovlivňuje. Budeme tedy testovat, zda data následků nehod pro  
silný vliv alkoholu patří do stejného rozložení jako ta pro slabý vliv  
alkoholu / bez alkoholu.

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency


Nejdříve si připravíme dataset. Potřebujeme rozdělit vlivy  
alkoholu a vážné následky na silný/slabý a vážné/lehké.

Načteme dataframe

In [ ]:
df = pd.read_pickle("accidents.pkl.gz")
df.head()

Nechceme nehody pod vlivem drog

In [ ]:
print(df.shape)
df = df.loc[(df["p11"] != 5) & (df["p11"] != 6)]
print(df.shape)

Přidáme pole určující slabý/silný vliv a lehké/těžké následky

In [ ]:
df["severe consequences"] = df["p13a"] + df["p13b"] > 0
df["too much alcohol"] = df["p11"] >= 7
df.head()

Vytvoříme si kontingenční tabulku, nad kterou se bude provádět test

In [ ]:
ct = pd.crosstab(df["severe consequences"], df["too much alcohol"])
ct

Nyní máme připravená data. Stačí už jen provést test.

In [ ]:
chi2_contingency(ct)

P-hodnota 3.5960593458701547e-122 je extrémně blízká nule, tedy menší  
než 0.05. Z toho vyplývá, že je téměř nulová pravděpodobnost, že by  
data pocházela ze stejného rozložení. Proto zamítáme hypotézu, že by  
silný vliv alkoholu neměl dopad na vážnost následků nehody. Tím se s 95%   
jistotou dokazuje, že silný vliv alkoholu má dopad na vážnost následků nehody.

Hypotéza ještě zmiňuje slovíčko "častěji". Test dobré shody jenom prokáže,  
jestli existuje korelace nebo nikoliv. Ne jestli více/méně, na kterou  
stranu. To lze snadno ověřit ze statistiky.

In [ ]:
high_alcohol_and_severe = df[df["too much alcohol"] & df["severe consequences"]].shape[0]
low_alcohol_and_severe = df[(df["too much alcohol"] == False) & df["severe consequences"]].shape[0]
total_high_alcohol = df[df["too much alcohol"]].shape[0]
total_low_alcohol = df[df["too much alcohol"] == False].shape[0]
print("Pravděpodobnost vážných následků při")
print("- silném vlivu: {:.2f}%".format(100 * high_alcohol_and_severe / total_high_alcohol))
print("- slabém/žádném vlivu: {:.2f}%".format(100 * low_alcohol_and_severe / total_low_alcohol))

Jak je vidět, při silném vlivu alkoholu dochází více než 2x častěji k vážným  
následkům. Z toho plyne, že jsme na správné straně hypotézy, tedy že se to  
děje častěji. Tato statistika sama za sebe naznačuje pravdivost hypotézy i  
bez testu a jeho výsledek potvrzuje. Proto lze prohlásit, že s 95% jistotou  
platí, že pokud viník nehody byl pod silným vlivem alkoholu, došlo častěji  
k těžkým zdravotním následkům.